In [ ]:
from simple_settings import LazySettings

settings = LazySettings('settings.development')

# display

In [ ]:
import socketio
import requests
from os import environ as env
import requests as rq
import cv2
import numpy as np
from matplotlib import pyplot as plt


class Display:
    def __init__(self, hostname):
        self._sio = socketio.Client()
        self.resolution = None

        @self._sio.on('resolution', namespace='/control')
        def on_resolution(w, h):
            print('resolution changed to:', w, h)
            self.resolution = (w, h)
            
        # connect to dispatch
        self._sio.connect('http://%s:5000' % hostname, namespaces=['/control'])
        self._sio.emit('handshake', namespace='/control', callback=self._ensure_ok)
        
    def highlight_pixel(self, x, y):
        self._sio.emit('highlight', data=(x, y), namespace='/control', callback=self._ensure_ok)
        
    def show_image(self, img):
        # fit target resolution
        img_out = cv2.resize(img, self.resolution)
        
        # send the image over sio
        image_payload = {
            'image': img_out.tobytes(),
            'width': img_out.shape[0],
            'height': img_out.shape[1]
        }
        self._sio.emit('show_image', data=image_payload, namespace='/control', callback=self._ensure_ok)
        
    def show_image_from_url(self, url):
        # fetch the image
        img = self._read_image_from_url(url)
        
        self.show_image(img)
        
    def _ensure_ok(self, resp):
        assert resp == 'ok', 'expected ack! got %s' % str(resp)
        
    def _read_image_from_url(self, url):
        r = rq.get(url)
        assert r.ok, 'Failed to fetch the image'
        img_str = r.content

        img_arr = np.frombuffer(img_str, np.uint8)

        img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
        assert img is not None, 'Failed to read the image'
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(img);

        return img

In [ ]:
display = Display(settings.DISPLAY_HOST)

In [ ]:
display.highlight_pixel(5, 6)

In [ ]:
display.show_image_from_url('https://t4.rbxcdn.com/05b0ee639de3e3e9b40dcbbad11a78b8')

# camera

In [ ]:
import cv2
from matplotlib import pyplot as plt
import threading, queue
from os import environ as env


class Camera:
    def __init__(self, hostname):
        # connect to RTSP server
        # ref: https://medium.com/beesightsoft/opencv-python-connect-to-android-camera-via-rstp-9eb78e2903d5
        env["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;udp"
        url = 'rtsp://%s:5554/camera' % hostname
        self._vcap = cv2.VideoCapture(url, cv2.CAP_FFMPEG)
        assert self._vcap.isOpened(), 'failed to open the RTSP stream:' + url
        
        # spawn a zero-buffer capture thread
        # ref: https://stackoverflow.com/a/54755738/154970
        self._q = queue.Queue()
        self._run = True
        self._t = threading.Thread(target=self._reader)
        self._t.daemon = True
        self._t.start()
        
    def close(self):
        print('stopping...')
        self._run = False
        self._t.join()
        
    def _reader(self):
        print('started the capture thread')
        while self._run:
            ok, img = self._vcap.read()
            if not ok:
                print('failed to read the stream')
                break
            if not self._q.empty():
                try:
                    self._q.get_nowait()
                except Queue.Empty:
                    pass
            self._q.put(img)
        print('shutting down the capture thread')
        
    def capture(self):
        img = self._q.get()
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
camera = Camera(settings.CAMERA_HOST)

In [ ]:
img = camera.capture()
plt.imshow(img);

In [ ]:
camera.close()